In [1]:
# Import Statements
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats
import copy
import math

# Setting to display all columns of dataframe
pd.set_option('display.max_columns', 30)

# Constants
QUEUE_BOUND = 3
SEARCH_MAX_ITER = 3

# Load countries
def load_countries():
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

# Load data methods
def load_up():
    
    #load and create df
    countryList = load_countries()
    root_country_df = pd.DataFrame(countryList).transpose()
    
    return root_country_df

def get_country_names(df):
    return df.index.to_list()

# Load dataframes
country_df = load_up()
root_country_df = load_up()
COUNTRY_NAMES = get_country_names(country_df)

# Maslow
L_ONE_RESOURCES = {'food': 1, 'water': 1}
L_TWO_RESOURCES = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 2, 'potential fossil energy': 1} 
L_THREE_RESOURCES = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
L_FOUR_RESOURCES = {'children': 2.5, 'renewable energy': 1}
L_FIVE_RESOURCES = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}

# List of levels
LEV_LIST = [L_ONE_RESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]

# Level Function
def leveldf(df, country, level, level_function_accessor):
    
    levelSat = False
    marUtility = False
    wasteRate = False
    mult = []
    average = 0

    for key, value in level_function_accessor[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = True
            
        #marginal utility
        if countryVal > value*3:
            marUtility = True
            
            if countryVal > value * 6:
                wasteRate = True
                
    
    average = stats.mean(mult)
    
    if levelSat:
        average = average*0.01
        
    if marUtility:
        average = average*0.01
        
    if wasteRate:
        average = average*0.00001
        
    return average

# Population Modeling
def pop(df):
    for row in range(len(df)):
        values = df.iloc[row]
        popVal = values[0]
        BR = round(np.random.normal(loc=values[len(values) - 2], scale=1.0, size=None))
        DR = round(np.random.normal(loc=values[len(values) - 1], scale=1.0, size=None))
        df.iat[row, 0] = popVal + BR - DR
        if df.iat[row, 0] <= 0:
            df.iat[row, 0] = 1
            
# Maslow Function
def maslowLevelVals(df, country, level, level_function_accessor):
    
    maslowList = []
    
    ## change population
    pop(df)
    
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, level_function_accessor)
        maslowList.append(levValue)
            
    return maslowList

def maslowList(lst):
    sumVal = 0
    sumVal = sumVal + lst[0] * 0.10
    sumVal = sumVal + lst[1] * 10
    sumVal = sumVal + lst[2] * 100
    sumVal = sumVal + lst[3] * 1000
    return sumVal

def maslowVal(df, country, level, LEV_LIST):
    
    lst = maslowLevelVals(df, country, level, LEV_LIST)
    val = maslowList(lst)
    return val


# Transforms
HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM_TM = ['FARM', {'population': 1, 'land' : 25, 'water': 25}, {'food': 5, 'population': 1}]
LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]


def transform(df, country, transform_template):
    
    allowed = True
    
    #check if transform is possible
    for key in transform_template[1]:
        val = transform_template[1][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform_template[1]:
            val = transform_template[1][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform_template[2]:
            val = transform_template[2][key]
            df.loc[country, key] += val
    return allowed


# Transfers
FOOD_TR = ['FOOD', {'food': 5}]
WATER_TR = ['WATER', {'water': 5}]
TIMBER_TR = ['TIMBER', {'timber': 5}]
METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]

ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]

def transfer(df, country1, country2, transfer_template):

    allowed = True

    #check for validity
    
    if country1==country2:
        allowed = False

    for key in transfer_template[1]:
        val = transfer_template[1][key]
        if (df.loc[country1,key] - val < 0):
            allowed = False
            
    if(allowed):
        #remove resource from country 1
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country1, key] -= val

        #add resource to country 2
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country2, key] += val
            
    return allowed

# Trade
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False
    

# Node & Priority Queue Def
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.state_quality
    def getExutil(self):
        return self.exp_util
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
    def __lt__(self, other):
        return self.exp_util < other.exp_util
    def __str__(self):
        s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
        return s
    
class BoundedPriorityQueue:
    # can change bound
    def __init__(self, queue=list(), bound=QUEUE_BOUND):
        self.bound = bound
        self.queue = queue
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        if len(self.queue) >= 1:
            return pq.heappop(self.queue)[-1]
        print("no nodes on fringe")
    ##maybe delete    
    def items(self):
        return list(item for _,_, item in self.queue)
    def clear(self):
        self.queue.clear()
    def len(self):
        return len(self.queue)
    def getState(self, index):
        if len(self.queue) > index:
            return self.queue[index][1]
        print("index out of bound")
    def pState(self, index):
        if len(self.queue) > index:
            print(self.queue[index][1])
        print("index out of bound")
    def getPriority(self, index):
        if len(self.queue) > index:
            return self.queue[index][0]
        print("index out of bound")
    def popBest(self):
        if len(self.queue) >= 1:
            return pq.nlargest(1, self.queue)[0][1]
        print("no nodes on fringe")
    def copy(self):
        return BoundedPriorityQueue(self.queue.copy())


# Generate Successors
def generate_successors(parentNode, my_country, LEV_LIST):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for resource in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf().copy()
        
        if transform(new_df, my_country, resource):
            state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
            new_schedule = prevSched.copy()
            new_state = Node(state_qual, 0, new_schedule, new_df)
            new_state.sched.append("TRANSFORM: " + resource[0])
            new_state.exp_util = expected_utility_transform(new_state, my_country, LEV_LIST)
            successors.push(new_state.exp_util, new_state)


    for resource_1 in ALL_TEMPLATES_TRANSFER:
        for resource_2 in ALL_TEMPLATES_TRANSFER:
            for other_country in COUNTRY_NAMES:
                
                new_df = parentNode.getCountrydf().copy()
                
                if trade(new_df, my_country, other_country, resource_1, resource_2):
                    state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
                    state_qual_other = maslowVal(new_df, other_country, 5, LEV_LIST)
                    new_schedule = prevSched.copy()
                    new_state = Node(state_qual, 0, new_schedule, new_df)
                    trade_name = "TRADE: " + my_country + " GIVES " + resource_1[0] + ", " + other_country + " GIVES " + resource_2[0]
                    new_state.sched.append(trade_name)
                    new_state.exp_util = expected_utility_trade(L, k, x_0, new_state, my_country, LEV_LIST, other_country)
                    successors.push(new_state.exp_util, new_state)
                    
    return successors

# Reward Functions
def undiscounted_reward(node, my_country,LEV_LIST):
    root_node_score = maslowVal(root_country_df, my_country, 5, LEV_LIST)
    cur_node_score = maslowVal(node.getCountrydf(), my_country, 5, LEV_LIST)
    return cur_node_score - root_node_score

GAMMA = 0.05

def discounted_reward(node, my_country, LEV_LIST):
    return (GAMMA ** len(node.getSched()))*undiscounted_reward(node, my_country, LEV_LIST)

L = 1.0
k = 1.0
x_0 = 0.0

def probability(L, k, node, other_country, LEV_LIST, x_0):
    return (L/(1+math.exp(-k*(discounted_reward(node, other_country, LEV_LIST)-x_0))))

COST_FAILURE = -0.05

def expected_utility_trade(L, k, x_0, node, my_country, LEV_LIST, other_country):
    prob = probability(L, k, node, other_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, other_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    other_country_utility = success + fail
    prob = probability(L, k, node, my_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, my_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    my_country_utility = success + fail
    total = (other_country_utility+my_country_utility)/2
    return total

PROB = 0.95
COST_FAILURE = -0.05
def expected_utility_transform(node, my_country, LEV_LIST):
    success = PROB * discounted_reward(node, my_country, LEV_LIST)
    fail = (1-PROB)*COST_FAILURE 
    return success + fail

# Search Function
def search(root_node, my_country,level_accessor):
    
    num = 1
    
    fringe = generate_successors(root_node, my_country, level_accessor)
    newfringe = BoundedPriorityQueue()
    for i in range(1, SEARCH_MAX_ITER):
        fringe_bound = fringe.len()
        for j in range(0,fringe_bound):
            successors = generate_successors(fringe.pop(), my_country, level_accessor)
            for k in range(0,successors.len()-1):
                newfringe.push(successors.getState(k).getExutil(),successors.getState(k))
                print("Explored Node: " + str(num))
                num = num + 1
        print("Level: " + str(i))
        print(newfringe.popBest())
        fringe = newfringe.copy()
        newfringe.clear()
    return fringe.popBest()


# Executable Code

In [2]:
state_qual = 1.2
exp_util = 3.0
sched = ['ROOT STATE']
test_node = Node(state_qual, exp_util, sched, country_df)
nodeBest = search(test_node, 'The Vale', LEV_LIST)
print(nodeBest)

Explored Node: 1
Explored Node: 2
Explored Node: 3
Explored Node: 4
Explored Node: 5
Explored Node: 6
Level: 1
Node with State Quality: 10.913423669512195
Expected Utility: -0.0007354683586301306
Schedule: ['ROOT STATE', 'TRANSFORM: ELECTRONICS']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              22.0   30.0   50.0   30.0    20.0     40.0   
Dinotopoia            10.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               28.0   40.0   60.0   20.0    76.0    100.0   
King's Landing       100.0  200.0  100.0   50.0    40.0     80.0   
The Vale              41.0  100.0   49.0  100.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale